## Keyword Based Labeling for text data

This solution helps create large training datasets without manually labeling them over weeks or months. It uses weak supervision approach and keyword based heuristics with the help of labeling functions (LFs) to assign labels to unlabeled training data and provide a clean labeled data as output. The labels are further enhanced using confidence learning methodologies. The output contains a CSV file consisting of the text, keyword based labels and enhanced clean labels. The solution is beneficial for obtaining automated clean class labels for input text datasets with less manual effort.

This sample notebook shows you how to execute Keyword Based Labeling for text data Algorithm from AWS Marketplace 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Keyword Based Data Labeling for text data. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Keyword Based Labeling for text data**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
algo_arn = "snorkle-keyword-training"

### 2. Prepare dataset

In [2]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:
* Input File name should be input_zip.zip
* The input data files must contain one csv file and one json file .
* For detailed instructions, please refer sample notebook and algorithm input details

#### B. Configure dataset

In [3]:
training_dataset="Input/input_zip.zip"

#### C. Upload datasets to Amazon S3

In [4]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "snorkle-keyword"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

## 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm. 

### 3.1 Set up environment

In [7]:
output_location = 's3://{}/snorkle-keyword/{}'.format(bucket, 'Output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [8]:
training_instance_type="ml.m5.4xlarge"

In [9]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="snorkle-keyword-algo",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

2023-04-25 04:19:45 Starting - Starting the training job...
2023-04-25 04:20:09 Starting - Preparing the instances for trainingProfilerReport-1682396385: InProgress
...
2023-04-25 04:20:42 Downloading - Downloading input data...
2023-04-25 04:21:09 Training - Downloading the training image.........
2023-04-25 04:22:29 Training - Training image download completed. Training in progress...2023-04-25 04:22:59.527758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 04:22:59.527793: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-25 04:23:01.032837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
202

#015 1/16 [>.............................] - ETA: 1:03 - loss: 0.8194 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/16 [==>...........................] - ETA: 58s - loss: 0.8221 - categorical_accuracy: 0.0000e+00 #010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015 1/16 [>.............................] - ETA: 1:00 - loss: 0.9214 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/16 [==>...........................] - ETA: 54s - loss: 0.9902 - categorical_accuracy: 0.0000e+00 #010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015 1/16 [>.............................] - ETA: 30s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/16 [==>...........................] - ETA: 16s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/16 [====>.........................] - ETA: 15s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 4/16 [======>.......................] - ETA: 14s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015

#015 1/16 [>.............................] - ETA: 1:00 - loss: 0.8912 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/16 [==>...........................] - ETA: 55s - loss: 0.9854 - categorical_accuracy: 0.0000e+00 #010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

...

#015 1/25 [>.............................] - ETA: 1:34 - loss: 1.7410 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/25 [=>............................] - ETA: 1:32 - loss: 1.7811 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015 1/25 [>.............................] - ETA: 1:38 - loss: 1.9680 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/25 [=>............................] - ETA: 1:30 - loss: 1.6751 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015 1/25 [>.............................] - ETA: 1:35 - loss: 1.3337 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/25 [=>............................] - ETA: 1:30 - loss: 1.5321 - categorical_accuracy: 0.0000e+00#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015 1/31 [..............................] - ETA: 3:47#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/31 [>.............................] - ETA: 34s #010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 3/31 [=>............................] - ETA: 33s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 4/31 [==>...........................] - ETA: 32s#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0


2023-04-25 04:45:33 Uploading - Uploading generated training model
2023-04-25 04:45:33 Completed - Training job completed
Training seconds: 1492
Billable seconds: 1492


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

## Note: Inferencing is done within training pipeline. Real time inference endpoint/batch transform job is not required.

### 3.3 Inspect the Output in S3

In [11]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)

In [12]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [ ]:
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

In [14]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output')

In [22]:
output_path = "output/output"
output = pd.read_csv(os.path.join(output_path, "output.csv"))
output=output[output.columns[1:]]
output.head(10)

,author,date,text,base_label,clean_label
0,Alessandro leite,2014-11-05T22:21:36,pls http://www10.vakinha.com.br/VaquinhaE.aspx...,1,1
1,Salim Tayara,2014-11-02T14:33:30,"if your like drones, plz subscribe to Kamal Ta...",1,1
2,Phuc Ly,2014-01-20T15:27:47,go here to check the views :3﻿,2,2
3,DropShotSk8r,2014-01-19T04:27:18,"Came here to check the views, goodbye.﻿",2,2
4,Giang Nguyen,2014-11-06T04:55:41,https://www.facebook.com/teeLaLaLa﻿,1,1
5,Malin Linford,2014-11-05T01:13:43,Hey guys please check out my new Google+ page ...,1,1
6,Lone Twistt,2013-11-28T17:34:55,Once you have started reading do not stop. If...,1,0
7,Олег Пась,2014-11-03T23:29:00,Plizz withing my channel ﻿,1,1
8,JD COKE,2014-11-08T02:24:02,"It's so hard, sad :( iThat little child Actor ...",1,0
9,zhichao wang,2013-11-29T02:13:56,i think about 100 millions of the views come f...,2,1


In [26]:
with open(os.path.join(output_path, "tagged_label.json"),"rb") as f:
    tagged_classes=json.load(f)
tagged_classes

{'HAM': 0, 'SPAM_1': 1, 'SPAM_2': 2}

### 4. Clean-up

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.